### 1- Import test data


In [233]:
from dotenv import load_dotenv
import os
import pprint

import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import pytz
import numpy as np


from pybit.unified_trading import HTTP

load_dotenv()


#connect to bYBIT

session = HTTP(
    testnet=False,
    demo=True,
    api_key=os.getenv('API_DEMO_KEY'),
    api_secret=os.getenv('API_DEMO_SECRET'),
)

### Bybit methods

In [242]:

def get_account_balance(ticker):
    # Retrieve account balance from Bybit
    response = session.get_wallet_balance(accountType="UNIFIED",coin=ticker)  

    
    if response['retCode'] == 0:
        balance = response['result']['list'][0]['coin'][0]['walletBalance']
        print(f'The balance of {ticker} is: {balance}')
        return balance
    else:
        print("Failed to retrieve account balance:", response['ret_msg'])
        return None


def fetch_market_data(symbol, interval, category):

    #Setup graph
    data = session.get_kline(symbol=symbol, interval=interval, category = category)
    df = pd.DataFrame(data['result']['list'])
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']

    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')
    df.set_index('Date', inplace=True)
    melbourne_tz = pytz.timezone('Australia/Melbourne')
    df.index = df.index.tz_localize('UTC').tz_convert(melbourne_tz)
    df.index = df.index.tz_localize(None)

    df = df[::-1]

    #add relevant indicators
    df['RSI'] = ta.rsi(df['Close'].astype(float), length = 14)
    df['Fast_EMA'] = ta.ema(df['Close'].astype(float), length=9)
    df['Slow_EMA'] = ta.ema(df['Close'].astype(float), length=30)

    df['ATR'] = ta.atr(df['High'].astype(float), df['Low'].astype(float),df['Close'].astype(float), length=7)

    bbands = ta.bbands(df['Close'].astype(float), length = 20, std = 2)
    df = df.join(bbands)


    backcandles = 6
    #df.reset_index(inplace=True, drop=True)
    df['EMA_SIGNAL'] = [ema_signal(df, i) if i >= backcandles - 1 else 0 for i in range(len(df))]
    df['TOTAL_SIGNAL'] = [total_signal(df, i) if i >= backcandles-1 else 0 for i in range(len(df))]


    df['pointpos'] = [
    float(row['Low']) - 500 if row['TOTAL_SIGNAL'] == 1 else #LONG
    float(row['High']) + 500 if row['TOTAL_SIGNAL'] == -1 else #SHORT
    None
    for _, row in df.iterrows()
]
    return df


def ema_signal(df, current_candle, backcandles=6 ):
    df_copy = df.copy()
    start = max(0, current_candle - backcandles+1) #starts at 0, or whatever candle we can reach . ADD 1 if we want to include current candle right?
    end = current_candle+1      #add 1 ehre too
    df_new = df_copy.iloc[start:end]

    if all(df_new['Fast_EMA'] > df_new['Slow_EMA']):
        return 1  # Uptrend
    elif all(df_new['Fast_EMA'] < df_new['Slow_EMA']):
        return -1  # Downtrend
    else:
        return 0  
     

def total_signal(df, current_candle, backcandles = 6):

    #if EMA signal is uptrend and we close under bollinger band lower, we return a BUY signal
    if (ema_signal(df, current_candle, backcandles)==1 and df['Close'].astype(float).iloc[current_candle]<=df['BBL_20_2.0'].iloc[current_candle]
    ):
        return 1
    
    
    if (ema_signal(df, current_candle, backcandles)==-1 and df['Close'].astype(float).iloc[current_candle]>=df['BBU_20_2.0'].iloc[current_candle]
    ):
        return -1
    return 0




### Retrieve market data

In [243]:
symbol = 'BTCUSDT'
interval = '5'
category = 'linear'
df = fetch_market_data(symbol,interval, category )
display(df[df['TOTAL_SIGNAL']!= 0].tail(30))




fig = go.Figure(data = go.Candlestick(x=df.index,
                                      open = df['Open'],
                                      high = df['High'],
                                      low = df['Low'],
                                      close = df['Close']))
#fig.update_layout(xaxis_rangeslider_visible=False)

fig.add_trace(go.Scatter(x=df.index, y=df['Fast_EMA'], line=dict(color='blue'), name='Fast EMA (9)'))
fig.add_trace(go.Scatter(x=df.index, y=df['Slow_EMA'], line=dict(color='red'), name='Slow EMA (21)'))

fig.add_trace(go.Scatter(x=df.index, y=df['BBU_20_2.0'], line=dict(color='green', width = 1), name='Upper Band'))
fig.add_trace(go.Scatter(x=df.index, y=df['BBL_20_2.0'], line=dict(color='orange', width = 1), name='Lower Band'))


fig.add_scatter(x= df.index, y=df['pointpos'], mode='markers', marker=dict(size=5, color = "MediumPurple"), name = "entry")
fig.update_layout(
    width=1200,  
    height=800  
)
fig.show()

,Open,High,Low,Close,Volume,Turnover,RSI,Fast_EMA,Slow_EMA,ATR,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0,EMA_SIGNAL,TOTAL_SIGNAL,pointpos
Date,,,,,,,,,,,,,,,,,,
2025-01-21 15:30:00,102685.3,102685.3,101976.9,102077.9,737.955,75429918.0578,39.683164,102598.905577,102421.325709,376.698740,102270.333696,102756.370,103242.406304,0.945997,-0.197962,1,1,101476.9
2025-01-21 17:55:00,101998.7,102171.7,101975.8,102161.9,432.711,44179844.7128,56.550206,101816.080544,101886.928417,302.371785,101251.894133,101699.640,102147.385867,0.880526,1.016208,-1,-1,102671.7
2025-01-22 01:35:00,104241,104252,103244.3,103483.8,3891.659,403555942.4519,36.074565,104429.660639,104358.323367,494.481181,103880.300252,104561.515,105242.729748,1.302993,-0.291024,1,1,102744.3


### Backtesting